Data

Load data

In [20]:
import nbinteract as nbi
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
# %matplotlib qt

In [21]:
print("Importing data")

df = pd.read_excel("data.xlsx",header =4)
df.drop(["Arr","Dep"], axis=1, inplace=True)
df.columns = ["Date","Train_Number","Station","Arrival_Date_Time",
               "Departure_Date_Time"]

print("Cleaning data")

df.sort_values(by=['Train_Number','Date', 'Arrival_Date_Time'], inplace=True,na_position='first')
df["Dwell"] = df.Departure_Date_Time - df.Arrival_Date_Time
df['Runtime'] = df.Arrival_Date_Time - df.Departure_Date_Time.shift()
df.loc[df['Runtime'] > pd.Timedelta(.5,unit="D"),"Runtime"] = pd.NaT
df.loc[df['Runtime'] < pd.Timedelta(1,unit="s"),"Runtime"] = pd.NaT
df.reset_index(inplace = True,drop=True)

print("Extracting list data")

values = "Date","Train_Number","Station"
dd = {i:None for i in values}

for key in dd:
    dd[key] = list(df[key].unique())
    dd[key] = [x for x in dd[key] if str(x)!= 'nan']
    dd[key].sort()

print("Data lists extracted")

start_date = min(df['Date'])
end_date = max(df['Date'])
df.set_index(['Train_Number','Station','Date'],inplace = True)
df.sort_values(by=["Train_Number","Date",'Arrival_Date_Time'])
print(df.shape)

df.head()

Importing data
Cleaning data
Extracting list data
Data lists extracted
(69924, 4)


Arrival_Date_Time Departure_Date_Time  \
Train_Number Station Date                                                 
701          BFD     2018-03-01                 NaT 2018-03-01 06:00:00   
             WAC     2018-03-01 2018-03-01 06:25:00 2018-03-01 06:27:00   
             COC     2018-03-01 2018-03-01 07:10:00 2018-03-01 07:11:00   
             HNF     2018-03-01 2018-03-01 07:31:00 2018-03-01 07:33:00   
             FNO     2018-03-01 2018-03-01 08:14:00 2018-03-01 08:16:00   

                                   Dwell  Runtime  
Train_Number Station Date                          
701          BFD     2018-03-01      NaT      NaT  
             WAC     2018-03-01 00:02:00 00:25:00  
             COC     2018-03-01 00:01:00 00:43:00  
             HNF     2018-03-01 00:02:00 00:20:00  
             FNO     2018-03-01 00:02:00 00:41:00

In [41]:
options = {
    'title': "Histogram"
}


def hist_response(TrainNumber,Station,RunType,Unit):
    global df
    out = df.xs(
        (TrainNumber, Station),
        level=[0, 'Station'])[RunType].dt.seconds.sort_index()
    units = {
        "Mins": 60,
        "Seconds": 1,
    }
    out = out/units[Unit]
    return(out)


nbi.hist(
    hist_response,
    TrainNumber = dd["Train_Number"],
    Station = dd["Station"],
    RunType = ["Runtime","Dwell"],
    Unit = ["Mins","Seconds"]
#     options = options
)

In [ ]:
nbi.publish(spec, nb_name, template='full', save_first=True)